### Initialize

In [1]:
import requests
import json
import os
import re
import datetime
import pickle
import socket
import base64
import pandas as pd
from tqdm import tqdm_notebook, tqdm
from time import sleep

In [2]:
MAX_COUNTS = 3
TIME_SLEEP = 1
DATA_PATH = './_data/rpl/'
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

### Get token

In [3]:
def string2base64(s):
    return base64.b64encode(s.encode('utf-8')).decode('ascii')
def get_token_data(log_url, creds, data):
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': 'Basic {}'.format(string2base64(creds))
    }
    r = requests.post(
        log_url, 
        headers=headers,
        data=data
    )
    try:
        token_data = r.json()
    except:
        print(r.text, '\n')
        token_data = {}
    return token_data

In [4]:
log_url = 'https://rfpl.sportand.me/authserver/oauth/token'
creds = 'rfsconsumer:K0Ka3jnfsd9afsd9llaD5gkss'
data = 'grant_type=client_credentials'
token_data = get_token_data(log_url, creds, data)
print(token_data)
token = token_data['access_token']
print('token: ', token)

{'access_token': '02ed7183-0524-41ce-a2f4-0ad57d62987c', 'token_type': 'bearer', 'expires_in': 503851, 'scope': 'read'}
token:  02ed7183-0524-41ce-a2f4-0ad57d62987c


### Utils

In [5]:
def get_data(url, token, limit=100, offset=0, max_counts=5, time_sleep=5):
    offset = offset
    data = []
    count = 0
    while True:
        print('url: ', url, ' | processing offset from: ', offset)
        params = {'access_token': token, 'limit': limit, 'offset': offset}
        r = requests.get(url, params=params)
        if r.json()['success']:
            if not r.json()['errors']:
                if (len(r.json()['data']) == 0):
                    break
                elif isinstance(r.json()['data'], dict):
                    data = r.json()['data']
                    break
                else:
                    data.extend(r.json()['data'])
                    offset += len(r.json()['data'])
            else:
                print('errors: ', r.json()['errors'])
                count += 1
                sleep(time_sleep)
        else:
            print('success: ', r.json()['success'], ' | errors: ', r.json()['errors'])
            count += 1
            sleep(time_sleep)
        if count >= max_counts:
            break
    return data

### By person

In [6]:
by_person_dict = {
    'clubofficialperson_data': 'https://rfpl.sportand.me/api/clubofficialperson',
    'officialperson_data': 'https://rfpl.sportand.me/api/officialperson',
    'person_data': 'https://rfpl.sportand.me/api/person',
    'player_data': 'https://rfpl.sportand.me/api/player',
    'teamrepresentative_data': 'https://rfpl.sportand.me/api/teamrepresentative'
}

In [7]:
for name, url in by_person_dict.items():
    print('processing: ', name)
    data = get_data(url, token, limit=1000, offset=0, max_counts=MAX_COUNTS, time_sleep=TIME_SLEEP)
    file_name = '{}{}.txt'.format(DATA_PATH, name)
    with open(file_name, 'w') as file:
        json.dump(data, file)

processing:  clubofficialperson_data
url:  https://rfpl.sportand.me/api/clubofficialperson  | processing offset from:  0
url:  https://rfpl.sportand.me/api/clubofficialperson  | processing offset from:  1000
url:  https://rfpl.sportand.me/api/clubofficialperson  | processing offset from:  1206
processing:  officialperson_data
url:  https://rfpl.sportand.me/api/officialperson  | processing offset from:  0
url:  https://rfpl.sportand.me/api/officialperson  | processing offset from:  1000
url:  https://rfpl.sportand.me/api/officialperson  | processing offset from:  1117
processing:  person_data
url:  https://rfpl.sportand.me/api/person  | processing offset from:  0
url:  https://rfpl.sportand.me/api/person  | processing offset from:  1000
url:  https://rfpl.sportand.me/api/person  | processing offset from:  2000
url:  https://rfpl.sportand.me/api/person  | processing offset from:  3000
url:  https://rfpl.sportand.me/api/person  | processing offset from:  4000
url:  https://rfpl.sportand.m

### By clubs and teams

In [8]:
by_clubs_teams_dict = {
    'clubs': 'https://rfpl.sportand.me/api/club',
    'teams': 'https://rfpl.sportand.me/api/team'
}

In [9]:
for name, url in by_clubs_teams_dict.items():
    print('processing: ', name)
    data = get_data(url, token, limit=1000, offset=0, max_counts=MAX_COUNTS, time_sleep=TIME_SLEEP)
    file_name = '{}{}.txt'.format(DATA_PATH, name)
    with open(file_name, 'w') as file:
        json.dump(data, file)

processing:  clubs
url:  https://rfpl.sportand.me/api/club  | processing offset from:  0
url:  https://rfpl.sportand.me/api/club  | processing offset from:  253
processing:  teams
url:  https://rfpl.sportand.me/api/team  | processing offset from:  0
url:  https://rfpl.sportand.me/api/team  | processing offset from:  239


### By tournaments

In [10]:
by_tournaments_dict = {
    'competitions': [
        'https://rfpl.sportand.me/api/competition', 
        'tournaments' #Идентификатор соревнования
    ], 
    'seasons': [
        'https://rfpl.sportand.me/api/season', 
        'applications', #Идентификатор сезона
        'tournaments' #Идентификатор сезона
    ], 
    'tournaments': [
        'https://rfpl.sportand.me/api/tournament', 
        'matches', #Идентификатор турнира
        'teams', #Идентификатор турнира
        'stages', #Идентификатор турнира
    ] 
}

In [11]:
for name, url_list in by_tournaments_dict.items():
    print('processing: ', name)
    data = get_data(url_list[0], token, limit=1000, offset=0, max_counts=MAX_COUNTS, time_sleep=TIME_SLEEP)
    file_name = '{}{}.txt'.format(DATA_PATH, name)
    with open(file_name, 'w') as file:
        json.dump(data, file)
    dir_name = '{}/{}'.format(DATA_PATH, name)
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    for item in data:
        id_item = item['id']
        for num_url_id in range(1, len(url_list)):
            print('processing: ', name, ' | sub: ', url_list[num_url_id], 'id: ', id_item)
            id_url = '{}/{}/{}'.format(url_list[0], id_item, url_list[num_url_id])
            id_data = get_data(id_url, token, limit=1000, offset=0, max_counts=MAX_COUNTS, time_sleep=TIME_SLEEP)
            file_name = '{}/id{}_{}.txt'.format(dir_name, id_item, url_list[num_url_id])
            with open(file_name, 'w') as file:
                json.dump(id_data, file)

processing:  competitions
url:  https://rfpl.sportand.me/api/competition  | processing offset from:  0
url:  https://rfpl.sportand.me/api/competition  | processing offset from:  6
processing:  competitions  | sub:  tournaments id:  1
url:  https://rfpl.sportand.me/api/competition/1/tournaments  | processing offset from:  0
url:  https://rfpl.sportand.me/api/competition/1/tournaments  | processing offset from:  89
processing:  competitions  | sub:  tournaments id:  2
url:  https://rfpl.sportand.me/api/competition/2/tournaments  | processing offset from:  0
url:  https://rfpl.sportand.me/api/competition/2/tournaments  | processing offset from:  89
processing:  competitions  | sub:  tournaments id:  4
url:  https://rfpl.sportand.me/api/competition/4/tournaments  | processing offset from:  0
url:  https://rfpl.sportand.me/api/competition/4/tournaments  | processing offset from:  89
processing:  competitions  | sub:  tournaments id:  5
url:  https://rfpl.sportand.me/api/competition/5/tourna

url:  https://rfpl.sportand.me/api/tournament/81/stages  | processing offset from:  1
processing:  tournaments  | sub:  matches id:  83
url:  https://rfpl.sportand.me/api/tournament/83/matches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournament/83/matches  | processing offset from:  240
processing:  tournaments  | sub:  teams id:  83
url:  https://rfpl.sportand.me/api/tournament/83/teams  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournament/83/teams  | processing offset from:  16
processing:  tournaments  | sub:  stages id:  83
url:  https://rfpl.sportand.me/api/tournament/83/stages  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournament/83/stages  | processing offset from:  30
processing:  tournaments  | sub:  matches id:  80
url:  https://rfpl.sportand.me/api/tournament/80/matches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournament/80/matches  | processing offset from:  4
processing:  tournament

url:  https://rfpl.sportand.me/api/tournament/53/teams  | processing offset from:  2
processing:  tournaments  | sub:  stages id:  53
url:  https://rfpl.sportand.me/api/tournament/53/stages  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournament/53/stages  | processing offset from:  1
processing:  tournaments  | sub:  matches id:  58
url:  https://rfpl.sportand.me/api/tournament/58/matches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournament/58/matches  | processing offset from:  240
processing:  tournaments  | sub:  teams id:  58
url:  https://rfpl.sportand.me/api/tournament/58/teams  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournament/58/teams  | processing offset from:  16
processing:  tournaments  | sub:  stages id:  58
url:  https://rfpl.sportand.me/api/tournament/58/stages  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournament/58/stages  | processing offset from:  33
processing:  tournaments  |

processing:  tournaments  | sub:  matches id:  32
url:  https://rfpl.sportand.me/api/tournament/32/matches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournament/32/matches  | processing offset from:  31
processing:  tournaments  | sub:  teams id:  32
url:  https://rfpl.sportand.me/api/tournament/32/teams  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournament/32/teams  | processing offset from:  32
processing:  tournaments  | sub:  stages id:  32
url:  https://rfpl.sportand.me/api/tournament/32/stages  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournament/32/stages  | processing offset from:  5
processing:  tournaments  | sub:  matches id:  25
url:  https://rfpl.sportand.me/api/tournament/25/matches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournament/25/matches  | processing offset from:  240
processing:  tournaments  | sub:  teams id:  25
url:  https://rfpl.sportand.me/api/tournament/25/teams  | pro

url:  https://rfpl.sportand.me/api/tournament/10/matches  | processing offset from:  0
success:  False  | errors:  [{'errorText': 'No row with the given identifier exists: [me.sportand.rfpl.model.Stadium#15]'}]
url:  https://rfpl.sportand.me/api/tournament/10/matches  | processing offset from:  0
success:  False  | errors:  [{'errorText': 'No row with the given identifier exists: [me.sportand.rfpl.model.Stadium#15]'}]
processing:  tournaments  | sub:  teams id:  10
url:  https://rfpl.sportand.me/api/tournament/10/teams  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournament/10/teams  | processing offset from:  32
processing:  tournaments  | sub:  stages id:  10
url:  https://rfpl.sportand.me/api/tournament/10/stages  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournament/10/stages  | processing offset from:  13
processing:  tournaments  | sub:  matches id:  5
url:  https://rfpl.sportand.me/api/tournament/5/matches  | processing offset from:  0
ur

In [12]:
id_tour_list = []
dir_name = '{}/tournaments'.format(DATA_PATH, name)
for file_name in [x for x in os.listdir(dir_name) if 'stages' in x]:
    with open('{}/{}'.format(dir_name, file_name), 'r') as file:
        data = json.load(file)
    id_tour_list.extend([x['id'] for x in data])
print('total ids tours: ', len(id_tour_list), ' | unique ids: ', len(set(id_tour_list)))
dir_name = '{}/{}'.format(DATA_PATH, 'tours')
if not os.path.exists(dir_name):
    os.makedirs(dir_name)
url_tours = 'https://rfpl.sportand.me/api/tournamentstage'
for id_tour in id_tour_list:
    id_url = '{}/{}'.format(url_tours, id_tour)
    id_data = get_data(id_url, token, limit=1000, offset=0, max_counts=MAX_COUNTS, time_sleep=TIME_SLEEP)
    file_name = '{}/id{}.txt'.format(dir_name, id_tour, url_list[num_url_id])
    with open(file_name, 'w') as file:
        json.dump(id_data, file)

total ids tours:  1467  | unique ids:  1467
url:  https://rfpl.sportand.me/api/tournamentstage/1533  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1534  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1535  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1536  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1537  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1538  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1539  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1540  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1541  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1542  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1543  | processing offset from:  0
url:  http

url:  https://rfpl.sportand.me/api/tournamentstage/933  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/934  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/935  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/936  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/937  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/938  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/939  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/940  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/941  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/942  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/943  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/944  | process

url:  https://rfpl.sportand.me/api/tournamentstage/1396  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1397  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1398  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1399  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1400  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1401  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1402  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1403  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1404  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1405  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1013  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/101

url:  https://rfpl.sportand.me/api/tournamentstage/1603  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1604  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1605  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1606  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1607  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1675  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1674  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1673  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1672  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1671  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1670  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/166

url:  https://rfpl.sportand.me/api/tournamentstage/1355  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1356  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1357  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1358  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1359  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1360  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1361  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1362  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1363  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1364  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1889  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/189

url:  https://rfpl.sportand.me/api/tournamentstage/214  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/215  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/216  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/217  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/218  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/219  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/220  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/221  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2141  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2142  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2143  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2144  | pro

url:  https://rfpl.sportand.me/api/tournamentstage/2434  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2435  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2436  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2437  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2438  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2439  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1513  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1516  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1519  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/1520  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/197  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2403

url:  https://rfpl.sportand.me/api/tournamentstage/2574  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2575  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2576  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2577  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2578  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2579  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2580  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2581  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2582  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2583  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/2584  | processing offset from:  0
url:  https://rfpl.sportand.me/api/tournamentstage/258

### By applications

In [13]:
by_applications_dict = {
    'applications': [
        'https://rfpl.sportand.me/api/application', 
        'coaches', #Идентификатор заявки клуба
        'heads', #Идентификатор заявки клуба
        'players' #Идентификатор заявки клуба
    ]
}

In [14]:
for name, url_list in by_applications_dict.items():
    print('processing: ', name)
    data = get_data(url_list[0], token, limit=1000, offset=0, max_counts=MAX_COUNTS, time_sleep=TIME_SLEEP)
    file_name = '{}{}.txt'.format(DATA_PATH, name)
    with open(file_name, 'w') as file:
        json.dump(data, file)
    dir_name = '{}/{}'.format(DATA_PATH, name)
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    for item in data:
        id_item = item['id']
        for num_url_id in range(1, len(url_list)):
            print('processing: ', name, ' | sub: ', url_list[num_url_id], 'id: ', id_item)
            id_url = '{}/{}/{}'.format(url_list[0], id_item, url_list[num_url_id])
            id_data = get_data(id_url, token, limit=1000, offset=0, max_counts=MAX_COUNTS, time_sleep=TIME_SLEEP)
            file_name = '{}/id{}_{}.txt'.format(dir_name, id_item, url_list[num_url_id])
            with open(file_name, 'w') as file:
                json.dump(id_data, file)

processing:  applications
url:  https://rfpl.sportand.me/api/application  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application  | processing offset from:  745
processing:  applications  | sub:  coaches id:  705
url:  https://rfpl.sportand.me/api/application/705/coaches  | processing offset from:  0
processing:  applications  | sub:  heads id:  705
url:  https://rfpl.sportand.me/api/application/705/heads  | processing offset from:  0
processing:  applications  | sub:  players id:  705
url:  https://rfpl.sportand.me/api/application/705/players  | processing offset from:  0
processing:  applications  | sub:  coaches id:  706
url:  https://rfpl.sportand.me/api/application/706/coaches  | processing offset from:  0
processing:  applications  | sub:  heads id:  706
url:  https://rfpl.sportand.me/api/application/706/heads  | processing offset from:  0
processing:  applications  | sub:  players id:  706
url:  https://rfpl.sportand.me/api/application/706/players  | process

processing:  applications  | sub:  heads id:  731
url:  https://rfpl.sportand.me/api/application/731/heads  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/731/heads  | processing offset from:  4
processing:  applications  | sub:  players id:  731
url:  https://rfpl.sportand.me/api/application/731/players  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/731/players  | processing offset from:  51
processing:  applications  | sub:  coaches id:  737
url:  https://rfpl.sportand.me/api/application/737/coaches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/737/coaches  | processing offset from:  13
processing:  applications  | sub:  heads id:  737
url:  https://rfpl.sportand.me/api/application/737/heads  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/737/heads  | processing offset from:  6
processing:  applications  | sub:  players id:  737
url:  https://rfpl.sportand.me/api/app

url:  https://rfpl.sportand.me/api/application/687/heads  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/687/heads  | processing offset from:  5
processing:  applications  | sub:  players id:  687
url:  https://rfpl.sportand.me/api/application/687/players  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/687/players  | processing offset from:  75
processing:  applications  | sub:  coaches id:  717
url:  https://rfpl.sportand.me/api/application/717/coaches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/717/coaches  | processing offset from:  19
processing:  applications  | sub:  heads id:  717
url:  https://rfpl.sportand.me/api/application/717/heads  | processing offset from:  0
processing:  applications  | sub:  players id:  717
url:  https://rfpl.sportand.me/api/application/717/players  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/717/players  | processing offset from: 

url:  https://rfpl.sportand.me/api/application/702/players  | processing offset from:  43
processing:  applications  | sub:  coaches id:  715
url:  https://rfpl.sportand.me/api/application/715/coaches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/715/coaches  | processing offset from:  13
processing:  applications  | sub:  heads id:  715
url:  https://rfpl.sportand.me/api/application/715/heads  | processing offset from:  0
processing:  applications  | sub:  players id:  715
url:  https://rfpl.sportand.me/api/application/715/players  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/715/players  | processing offset from:  38
processing:  applications  | sub:  coaches id:  671
url:  https://rfpl.sportand.me/api/application/671/coaches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/671/coaches  | processing offset from:  38
processing:  applications  | sub:  heads id:  671
url:  https://rfpl.sportand.me

url:  https://rfpl.sportand.me/api/application/634/players  | processing offset from:  123
processing:  applications  | sub:  coaches id:  646
url:  https://rfpl.sportand.me/api/application/646/coaches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/646/coaches  | processing offset from:  29
processing:  applications  | sub:  heads id:  646
url:  https://rfpl.sportand.me/api/application/646/heads  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/646/heads  | processing offset from:  7
processing:  applications  | sub:  players id:  646
url:  https://rfpl.sportand.me/api/application/646/players  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/646/players  | processing offset from:  92
processing:  applications  | sub:  coaches id:  651
url:  https://rfpl.sportand.me/api/application/651/coaches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/651/coaches  | processing offset fr

...

url:  https://rfpl.sportand.me/api/application/407/players  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/407/players  | processing offset from:  20
processing:  applications  | sub:  coaches id:  408
url:  https://rfpl.sportand.me/api/application/408/coaches  | processing offset from:  0
processing:  applications  | sub:  heads id:  408
url:  https://rfpl.sportand.me/api/application/408/heads  | processing offset from:  0
processing:  applications  | sub:  players id:  408
url:  https://rfpl.sportand.me/api/application/408/players  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/408/players  | processing offset from:  31
processing:  applications  | sub:  coaches id:  139
url:  https://rfpl.sportand.me/api/application/139/coaches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/139/coaches  | processing offset from:  24
processing:  applications  | sub:  heads id:  139
url:  https://rfpl.sportand.me/

url:  https://rfpl.sportand.me/api/application/294/players  | processing offset from:  20
processing:  applications  | sub:  coaches id:  31
url:  https://rfpl.sportand.me/api/application/31/coaches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/31/coaches  | processing offset from:  13
processing:  applications  | sub:  heads id:  31
url:  https://rfpl.sportand.me/api/application/31/heads  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/31/heads  | processing offset from:  6
processing:  applications  | sub:  players id:  31
url:  https://rfpl.sportand.me/api/application/31/players  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/31/players  | processing offset from:  61
processing:  applications  | sub:  coaches id:  304
url:  https://rfpl.sportand.me/api/application/304/coaches  | processing offset from:  0
processing:  applications  | sub:  heads id:  304
url:  https://rfpl.sportand.me/api/applica

url:  https://rfpl.sportand.me/api/application/73/heads  | processing offset from:  0
processing:  applications  | sub:  players id:  73
url:  https://rfpl.sportand.me/api/application/73/players  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/73/players  | processing offset from:  57
processing:  applications  | sub:  coaches id:  179
url:  https://rfpl.sportand.me/api/application/179/coaches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/179/coaches  | processing offset from:  6
processing:  applications  | sub:  heads id:  179
url:  https://rfpl.sportand.me/api/application/179/heads  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/179/heads  | processing offset from:  6
processing:  applications  | sub:  players id:  179
url:  https://rfpl.sportand.me/api/application/179/players  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/179/players  | processing offset from:  20
p

url:  https://rfpl.sportand.me/api/application/157/players  | processing offset from:  67
processing:  applications  | sub:  coaches id:  2
url:  https://rfpl.sportand.me/api/application/2/coaches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/2/coaches  | processing offset from:  25
processing:  applications  | sub:  heads id:  2
url:  https://rfpl.sportand.me/api/application/2/heads  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/2/heads  | processing offset from:  5
processing:  applications  | sub:  players id:  2
url:  https://rfpl.sportand.me/api/application/2/players  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/2/players  | processing offset from:  70
processing:  applications  | sub:  coaches id:  328
url:  https://rfpl.sportand.me/api/application/328/coaches  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/328/coaches  | processing offset from:  8
processing: 

processing:  applications  | sub:  heads id:  98
url:  https://rfpl.sportand.me/api/application/98/heads  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/98/heads  | processing offset from:  5
processing:  applications  | sub:  players id:  98
url:  https://rfpl.sportand.me/api/application/98/players  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/98/players  | processing offset from:  52
processing:  applications  | sub:  coaches id:  305
url:  https://rfpl.sportand.me/api/application/305/coaches  | processing offset from:  0
processing:  applications  | sub:  heads id:  305
url:  https://rfpl.sportand.me/api/application/305/heads  | processing offset from:  0
processing:  applications  | sub:  players id:  305
url:  https://rfpl.sportand.me/api/application/305/players  | processing offset from:  0
url:  https://rfpl.sportand.me/api/application/305/players  | processing offset from:  17
processing:  applications  | sub:  coaches

### By matches

In [15]:
id_matches_list = []
dir_name = '{}/tournaments'.format(DATA_PATH, name)
for file_name in [x for x in os.listdir(dir_name) if 'matches' in x]:
    with open('{}/{}'.format(dir_name, file_name), 'r') as file:
        data = json.load(file)
    id_matches_list.extend([x['id'] for x in data])
print('total ids matches: ', len(id_matches_list), ' | unique ids: ', len(set(id_matches_list)))

total ids matches:  9037  | unique ids:  9037


In [19]:
dir_name = '{}/{}'.format(DATA_PATH, 'matches')
if not os.path.exists(dir_name):
    os.makedirs(dir_name)
url_matches = 'https://rfpl.sportand.me/api/match'
start_index = len(os.listdir(dir_name))
print('start from: ', start_index)
for id_match in id_matches_list[start_index:]:
    id_url = '{}/{}'.format(url_matches, id_match)
    id_data = get_data(id_url, token, limit=1000, offset=0, max_counts=MAX_COUNTS, time_sleep=TIME_SLEEP)
    file_name = '{}/id{}.txt'.format(dir_name, id_match)
    with open(file_name, 'w') as file:
        json.dump(id_data, file)

start from:  8207
url:  https://rfpl.sportand.me/api/match/11250  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11256  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11253  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11254  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11260  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11262  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11263  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11265  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11261  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11258  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11264  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11259  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11271  | processi

url:  https://rfpl.sportand.me/api/match/11475  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11479  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11478  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11480  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11477  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11481  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11476  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11482  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11489  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11484  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11486  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11488  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11490  | processing offset from:  0

url:  https://rfpl.sportand.me/api/match/11707  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11706  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11704  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11715  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11711  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11714  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11709  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11716  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11712  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11713  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11710  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11720  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11723  | processing offset from:  0

url:  https://rfpl.sportand.me/api/match/11992  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11993  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11994  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11995  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11996  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11997  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11998  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/11999  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/12000  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/12001  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/12002  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/12003  | processing offset from:  0
url:  https://rfpl.sportand.me/api/match/12004  | processing offset from:  0

### Dictionaries

In [20]:
by_dictionaries_dict = {
    'countries': 'https://rfpl.sportand.me/api/country',
    'redcardtypes': 'https://rfpl.sportand.me/api/redcardtype',
    'refereecategories': 'https://rfpl.sportand.me/api/refereecategory',
    'regions': 'https://rfpl.sportand.me/api/region',
    'stadiums': 'https://rfpl.sportand.me/api/stadium',
    'yellowcardtypes': 'https://rfpl.sportand.me/api/yellowcardtype'
}

In [21]:
for name, url in by_dictionaries_dict.items():
    print('processing: ', name)
    data = get_data(url, token, limit=1000, offset=0, max_counts=MAX_COUNTS, time_sleep=TIME_SLEEP)
    file_name = '{}{}.txt'.format(DATA_PATH, name)
    with open(file_name, 'w') as file:
        json.dump(data, file)

processing:  countries
url:  https://rfpl.sportand.me/api/country  | processing offset from:  0
url:  https://rfpl.sportand.me/api/country  | processing offset from:  112
processing:  redcardtypes
url:  https://rfpl.sportand.me/api/redcardtype  | processing offset from:  0
url:  https://rfpl.sportand.me/api/redcardtype  | processing offset from:  7
processing:  refereecategories
url:  https://rfpl.sportand.me/api/refereecategory  | processing offset from:  0
url:  https://rfpl.sportand.me/api/refereecategory  | processing offset from:  11
processing:  regions
url:  https://rfpl.sportand.me/api/region  | processing offset from:  0
url:  https://rfpl.sportand.me/api/region  | processing offset from:  263
processing:  stadiums
url:  https://rfpl.sportand.me/api/stadium  | processing offset from:  0
url:  https://rfpl.sportand.me/api/stadium  | processing offset from:  264
processing:  yellowcardtypes
url:  https://rfpl.sportand.me/api/yellowcardtype  | processing offset from:  0
url:  htt

### History

In [24]:
history_dict = {
    'history': 'https://rfpl.sportand.me/api/history'
}

In [25]:
for name, url in history_dict.items():
    print('processing: ', name)
    data = get_data(url, token, limit=1000, offset=0, max_counts=MAX_COUNTS, time_sleep=TIME_SLEEP)
    file_name = '{}{}.txt'.format(DATA_PATH, name)
    with open(file_name, 'w') as file:
        json.dump(data, file)

processing:  history
url:  https://rfpl.sportand.me/api/history  | processing offset from:  0
url:  https://rfpl.sportand.me/api/history  | processing offset from:  1000
url:  https://rfpl.sportand.me/api/history  | processing offset from:  2000
url:  https://rfpl.sportand.me/api/history  | processing offset from:  3000
url:  https://rfpl.sportand.me/api/history  | processing offset from:  4000
url:  https://rfpl.sportand.me/api/history  | processing offset from:  5000
url:  https://rfpl.sportand.me/api/history  | processing offset from:  6000
url:  https://rfpl.sportand.me/api/history  | processing offset from:  7000
url:  https://rfpl.sportand.me/api/history  | processing offset from:  8000
url:  https://rfpl.sportand.me/api/history  | processing offset from:  9000
url:  https://rfpl.sportand.me/api/history  | processing offset from:  10000
url:  https://rfpl.sportand.me/api/history  | processing offset from:  11000
url:  https://rfpl.sportand.me/api/history  | processing offset from

In [28]:
data[:5]

[{'id': 17019,
  'actionDate': '2019-11-08_18:10:48',
  'modelType': 'TeamRepresentative',
  'modelId': 7861,
  'model': 'Неучев Игорь Николаевич 27.10.1974',
  'action': 'CREATE',
  'triggerModelType': 'TeamRepresentative'},
 {'id': 17018,
  'actionDate': '2019-11-08_17:58:48',
  'modelType': 'Match',
  'modelId': 11967,
  'model': 'Первенство России молодёжных (U-19) команд 2019-20 16 Ростов-М - Тамбов-М',
  'action': 'EDIT',
  'triggerModelType': 'YellowCard'},
 {'id': 17017,
  'actionDate': '2019-11-08_16:45:48',
  'modelType': 'Match',
  'modelId': 11966,
  'model': 'Первенство России молодёжных (U-19) команд 2019-20 16 Спартак-Москва-М - Крылья Советов-М',
  'action': 'EDIT',
  'triggerModelType': 'Match'},
 {'id': 17016,
  'actionDate': '2019-11-08_16:17:34',
  'modelType': 'FootballPlayer',
  'modelId': 26941,
  'model': 'Варданян Эрик  07.06.1998',
  'action': 'CREATE',
  'triggerModelType': 'FootballPlayer'},
 {'id': 17015,
  'actionDate': '2019-11-08_16:13:01',
  'modelType'